<a href="https://colab.research.google.com/github/mapcrafter2048/Literature-Review-Generator-ML-17/blob/main/LRG_using_Hugging_Face_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.10

In [2]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install .
!pip install huggingface-hub
!pip install keras_nlp
!pip install datasets
!pip install nltk
!pip install rouge-score

Cloning into 'transformers'...
remote: Enumerating objects: 220764, done.
remote: Counting objects: 100% (26290/26290), done.
remote: Compressing objects: 100% (2169/2169), done.
remote: Total 220764 (delta 25493), reused 24316 (delta 24055), pack-reused 194474
Receiving objects: 100% (220764/220764), 232.24 MiB | 14.54 MiB/s, done.
Resolving deltas: 100% (160850/160850), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 12.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.44.0.dev0-py3-none-any.whl size=9404289 sha256=520fec62352638dbf25b70db902da050a1198e34398f0a5c3e4b9a5299944780
  Stored in directory: /tmp/pip-ephem-wheel-cache-wjczv8wa/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting unins

In [3]:
import os
import logging
import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.get_logger().setLevel(logging.ERROR)

In [4]:
TRAIN_TEST_SPLIT = 0.1
MAX_INPUT_LENGTH = 1024
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 8
MAX_EPOCHS = 2
MODEL_CHECKPOINT = "t5-small"

In [22]:
from datasets import load_dataset

data = load_dataset("ccdv/pubmed-summarization", "document", split="train")

In [23]:
from datasets import load_dataset

data = load_dataset("ccdv/pubmed-summarization", "section", split="train")

In [24]:
print(data)

Dataset({
    features: ['article', 'abstract'],
    num_rows: 119924
})


In [25]:
print(data[0])

{'article': "a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries . \n in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively . \n the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% . \n anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight . \n snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states \n there are also some reports regarding school feeding programs in developi

In [26]:
data = data.train_test_split(
    train_size=TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

In [9]:
!pip install --upgrade tf_keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: tf_keras
    Found existing installation: tf_keras 2.15.1
    Uninstalling tf_keras-2.15.1:
      Successfully uninstalled tf_keras-2.15.1


In [10]:
!pip show tensorflow
!pip show tf_keras

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, tensorflow-text, tf_keras
Name: tf_keras
Version: 2.17.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: tensorflow
Required-by: tensorflow-hub


In [11]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [29]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["abstract"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [30]:
tokenized_datasets = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/11992 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4112: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11993 [00:00<?, ? examples/s]

In [31]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [32]:
optimizer = 'adam'
model.compile(optimizer=optimizer)

In [33]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)

    result = {"RougeL": result["f1_score"]}

    return result


In [34]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

Epoch 1/2
1499/1499 [==============================] - ETA: 0s - loss: 2.3981

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


1499/1499 [==============================] - 1694s 1s/step - loss: 2.3981 - val_loss: 2.1369 - RougeL: 0.1141
Epoch 2/2
1499/1499 [==============================] - 1705s 1s/step - loss: 2.1752 - val_loss: 2.0802 - RougeL: 0.1201


In [35]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

article = """This comprehensive study explores the substantial impact of aerodynamic design modifications on fuel efficiency in commercial aircraft, an area of critical importance given the aviation industry's ongoing efforts to reduce operating costs and environmental impact. The research employs a multifaceted approach, combining advanced computational fluid dynamics (CFD) simulations with empirical data collected from commercial flights to provide a thorough analysis of how various aerodynamic features contribute to fuel efficiency improvements.

Initially, the study investigates the role of wing design, focusing on the effects of winglets, wing curvature, and wing aspect ratios. The CFD simulations reveal that incorporating advanced winglets significantly reduces drag by mitigating vortex-induced drag at the wingtips. The optimal wing curvature and aspect ratio configurations further enhance aerodynamic performance by improving lift-to-drag ratios. These findings are corroborated by flight data analysis, which shows that aircraft equipped with optimized wing designs experience a reduction in fuel consumption of approximately 8% compared to traditional designs.

In addition to wing design, the research examines the impact of fuselage shape and surface materials on aerodynamic efficiency. The study introduces several fuselage design modifications, including streamlined shapes and the use of composite materials with lower drag coefficients. The simulations indicate that a more streamlined fuselage shape, combined with advanced surface materials, reduces drag and enhances overall aerodynamic performance. Real-world flight tests confirm that these modifications contribute to a reduction in fuel consumption by an additional 4%, highlighting the significance of fuselage design in improving fuel efficiency.

The study also explores the potential for hybrid approaches that integrate both aerodynamic design enhancements and new materials technologies. By combining optimized wing designs with advanced fuselage materials, the research demonstrates that the cumulative effects of these improvements can achieve a total fuel savings of up to 12%. This figure represents a significant advancement in fuel efficiency, with potential financial implications for the aviation industry estimated in the millions of dollars annually for major airlines.

The research emphasizes the broader implications of these findings for the future of sustainable aviation. The demonstrated improvements in fuel efficiency not only contribute to reduced operational costs but also align with global environmental goals by decreasing greenhouse gas emissions. The study calls for continued investment in aerodynamic research and development, advocating for the adoption of innovative design practices and materials technologies to further enhance aircraft performance and sustainability.

In conclusion, this study provides a detailed analysis of how aerodynamic design innovations can lead to substantial improvements in fuel efficiency for commercial aircraft. The integration of advanced wing designs, streamlined fuselage shapes, and cutting-edge materials technologies offers a promising pathway to reducing fuel consumption and operational costs. As the aviation industry faces increasing pressure to adopt more sustainable practices, the insights gained from this research underscore the importance of ongoing advancements in aerodynamic design for achieving both economic and environmental benefits."""

summarizer(
    article,
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': "[ purpose ] this comprehensive study explores the substantial impact of aerodynamic design modifications on fuel efficiency in commercial aircraft , an area of critical importance given the aviation industry's ongoing efforts to reduce operating costs and environmental impact . [ subjects and methods ] the research employs a multifaceted approach combining advanced computational fluid dynamics (CFD) simulations with empirical data collected from commercial flights to provide a thorough analysis of how various aerodynamic features contribute to fuel efficiency improvements . Initially, the study investigates the role of wing design , focusing on the effects of "}]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import the necessary library
from transformers import TFT5ForConditionalGeneration

# Assuming 'model' is your TFT5ForConditionalGeneration model
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/summarisation_HF')